In [1]:
# 01_limpieza_datos.ipynb
import pandas as pd
import numpy as np


In [2]:
# 1. Cargar datos simulados
df = pd.read_csv('../data/datos_credito_simulados.csv')

print("Datos originales:", df.shape)



Datos originales: (1000, 16)


In [3]:
# Mapeo de Variables Ordinales (Donde el orden importa)
# -------------------------------
# 2. Revisión general
# -------------------------------
df.info()
df.head()

# -------------------------------
# 3. Validación de SCORE
# -------------------------------
scores_validos = ['AAA', 'AA', 'A', 'Rechazado']
df = df[df['score_interno'].isin(scores_validos)]

# Codificación ordinal del score (riesgo creciente)
map_score = {
    'AAA': 1,
    'AA': 2,
    'A': 3,
    'Rechazado': 4
}
df['score_ordinal'] = df['score_interno'].map(map_score)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   score_interno       1000 non-null   object 
 1   dias_mora_prom      1000 non-null   int64  
 2   edad                1000 non-null   int64  
 3   ingreso_mensual     1000 non-null   float64
 4   ventas_anuales      1000 non-null   float64
 5   segmento_credito    1000 non-null   object 
 6   producto            1000 non-null   object 
 7   monto_solicitado    1000 non-null   float64
 8   plazo_meses         1000 non-null   int64  
 9   garantia            1000 non-null   object 
 10  propiedad_completa  1000 non-null   int64  
 11  estado_civil        1000 non-null   object 
 12  estado_legal        1000 non-null   int64  
 13  tiene_garante       1000 non-null   int64  
 14  rastreo_instalado   1000 non-null   int64  
 15  riesgo_real         1000 non-null   int64  
dtypes: floa

In [4]:
# -------------------------------
# 4. Variable objetivo
# -------------------------------
df['riesgo_real'] = df['riesgo_real'].astype(int)



In [5]:
# -------------------------------
# 5. Variables categóricas
# -------------------------------
cols_categoricas = [
    'segmento_credito',
    'producto',
    'garantia',
    'estado_civil'
]

df = pd.get_dummies(df, columns=cols_categoricas, drop_first=True)


In [6]:
# -------------------------------
# 6. Eliminación de columnas no entrenables
# -------------------------------
df_modelo = df.drop(columns=[
    'score_interno'  # ya está representado en score_ordinal
])

In [7]:
# -------------------------------
# 7. Dataset final
# -------------------------------
df_modelo.info()
df_modelo.head()

# Guardar dataset limpio
df_modelo.to_csv("datos_credito_simulados_listos.csv", index=False)
print("¡Limpieza completada!")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   dias_mora_prom                 1000 non-null   int64  
 1   edad                           1000 non-null   int64  
 2   ingreso_mensual                1000 non-null   float64
 3   ventas_anuales                 1000 non-null   float64
 4   monto_solicitado               1000 non-null   float64
 5   plazo_meses                    1000 non-null   int64  
 6   propiedad_completa             1000 non-null   int64  
 7   estado_legal                   1000 non-null   int64  
 8   tiene_garante                  1000 non-null   int64  
 9   rastreo_instalado              1000 non-null   int64  
 10  riesgo_real                    1000 non-null   int64  
 11  score_ordinal                  1000 non-null   int64  
 12  segmento_credito_Consumo       1000 non-null   bo